In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
pd.set_option('display.max_columns', None)

In [ ]:
df = pd.read_csv('df_cleaned.csv') 
df

## Categorical columns

In [ ]:
categorical = df.select_dtypes(include=object)

In [ ]:
categorical.columns.tolist()

We will not focus on 'bookId', 'title', 'author', 'genres' and 'publishDate' columns because it as to many unique values.\
We keep 'author' and 'genres' columns for later.\
We add also the column 'awards' since there's only to values, 1 for 'yes' and 0 for 'no'.

In [ ]:
categorical = df[['series', 'language',
 'bookFormat',
 'publisher', 'awards']]

In [ ]:
for col in categorical.columns:
    display(categorical[col].value_counts())

### 1. Language

Most of the books are in English, all the others languages will be grouped in 'other'

In [ ]:
categorical['language'] = np.where(categorical['language'].isin(['English']),'English','Other')


In [ ]:
categorical['language'].value_counts()

### 2. Book format

Most of the book are paperback or hardcover, all the others formats will be grouped in 'other'

In [ ]:
categorical['bookFormat'] = np.where(categorical['bookFormat'].isin(['Paperback', 'Hardcover']),categorical['bookFormat'],'Other')
categorical['bookFormat'].value_counts()

### 3. Publisher

Since readers may be sensitive to the publisher we will take a closer look to the most 10 famous ones and group the others in 'Other'.

In [ ]:
publisher = pd.DataFrame(categorical['publisher'].value_counts())
publisher.index.tolist()

In [ ]:
categorical['publisher'] = np.where(categorical['publisher'].isin([
'Vintage',
 'HarperCollins',
 'Penguin Books',
 'Ballantine Books',
 'Bantam',
 'Createspace Independent Publishing Platform',
 'Pocket Books',
 'Avon',
 'Berkley',
 'Del Rey']),categorical['publisher'],'Other')
categorical['publisher'].value_counts()

## Numerical columns

In [ ]:
numerical = df.select_dtypes(include=np.number)
numerical = numerical.drop(columns=['awards','bbeScore', 'bbeVotes'])
numerical

## Outliers

In [ ]:
for col in numerical:
    numerical[[col]].boxplot()
    plt.show()

Only the column 'pages' seems to have problematic outliers, let's remove them 

In [ ]:
iqr = np.percentile(numerical['pages'],75) - np.percentile(numerical['pages'],25)
upper_limit = np.percentile(numerical['pages'],75) + 5*iqr
lower_limit = np.percentile(numerical['pages'],25) - 5*iqr
df_no_outliers = numerical[(numerical['pages']>lower_limit) & (numerical['pages']<upper_limit)].copy()
sns.displot(df_no_outliers['pages'])
plt.show()

In [ ]:
numerical['pages'] = df_no_outliers['pages']

In [ ]:
numerical[['pages']].boxplot()

### Now we have removed the outliers, let's check for correlations

In [ ]:
correlations_matrix = numerical.corr()
sns.heatmap(correlations_matrix, annot=True)

We can see that ratins are corralated with the percentage of liked books which can be explained by the fact that liked books are those with more than 2 stars.\


In [ ]:
for column in numerical:
    sns.distplot(numerical[column])
    plt.show()